In [4]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [5]:
!ls

box2mask.ipynb			     sort_df_for_ensemble.ipynb
check_preds.ipynb		     sub_4195_100_275_5000.csv
check_preds-parent.ipynb	     sub_4195_100_275.csv
eda.ipynb			     sub_4195_100_276.csv
ensemble_csv.ipynb		     sub_4195_20_100_5000.csv
ensemble.ipynb			     sub_4195_20_100.csv
ens_htc275_4619_od1000_0928_275.csv  submit.ipynb
filter_classes.ipynb		     submit-parent.ipynb
mask2box.ipynb			     top_classes.csv
merge_df.ipynb			     top_classes_level1.csv
od_convert_0928.csv		     top_classes_parent.csv
od_convert_0928_top1000.csv	     train_parent.ipynb


In [18]:
csv_files = [
            '../sub_htc275_lb4560_top100_iou03_mask42_lb4619.csv',
            'od_convert_0928_top1000_filter275_lowconf.csv',
            '../sub_htc_parent_0927am_lb0338.csv' #,
            #'sub_0902_3_top50.csv.zip',
            #'sub_parent_0903_1_top50.csv'
            ]

dfs = [pd.read_csv(x) for x in csv_files]

In [19]:
for df in dfs:
    df.PredictionString = df.PredictionString.fillna('')

In [20]:
for i in range(1, len(dfs)):
    dfs[i] = dfs[i].set_index('ImageID')
    dfs[i] = dfs[i].reindex(index=dfs[0]['ImageID'])
    dfs[i] = dfs[i].reset_index()

In [21]:
len(dfs[1].iloc[0].PredictionString)

5800

In [22]:
# lower conf for is model, not working
# dfs[1].PredictionString = dfs[1].PredictionString.map(lambda x: get_low_conf_pred_str(x))

In [23]:
dfs[0].head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0001552 eNodkuuyqjAMRl+p8Xqq4gyIMFt...
1,52a76f3fe21185bf,1024,683,/m/04dr76w 0.0074179 eNqlU+ly2jAQfiVrdYAduykxD...
2,65e9da7c04c8e4cd,1024,682,/m/04yx4 0.0002104 eNpdkltzokAQhf/S9FAWbGVrH1L...
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0006262 eNpFiUEKgDAMBL+0SSuCHgRBpJ...
4,0841dee573a875ca,1024,683,/m/05r655 0.0010084 eNp1kutOAjEQhV/pdLfcjMaoQZ...


In [24]:
#keep top 100
#dfs[0].PredictionString = dfs[0].PredictionString.map(lambda x: ' '.join(x.split(' ')[:1200]))

In [25]:
dfs[1].head(10)

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/01kb5b 0.00682 eNqLiUgwTbbOsDQYBaNgFAwFkJ4U...
1,52a76f3fe21185bf,1024,683,/m/0bjyj5 0.00669 eNpLS4q0DUkyCDc2GAWjYDiChEhb...
2,65e9da7c04c8e4cd,1024,682,/m/0fszt 0.00035 eNoLTDfKTjKINzIYBaNgFIyCUTAKR...
3,6d10fa21d93bd6cc,1024,682,/m/04yx4 0.00852 eNpLS822Sk4xSDcYBaNgBIOA3FgjA...
4,0841dee573a875ca,1024,683,/m/0342h 0.00693 eNqLyky0TDDLtjMYBaOANBAbl2MKA...
5,660329181edd20b4,1024,483,/m/0cnyhnx 0.00940 eNqLyc81yrKKNDEYBaMABILiIi0...
6,2dffa0787ba35f70,1024,545,/m/06m11 0.00545 eNpLTU+xzjcKsjMYBbhBdnK+EQC1F...
7,cb66703ecbd0b99a,1024,678,/m/079cl 0.00687 eNrLS8k3j7dNNzcYBcSDpLRAKwB17...
8,2f01f2c68316b5e5,1024,768,/m/01btn 0.00758 eNqLCQmzyLYMtTMYBaNgFIyCUUAMS...
9,36cfe3fa46f56ed3,768,768,/m/079cl 0.00491 eNoLSjBJSTewMRgFo2AUjIJRMNRBX...


In [26]:
x = None
for d in dfs:
    if x is None:
        x = d.PredictionString.map(lambda x: x.strip())
    else:
        x = x.str.cat(d.PredictionString.map(lambda x: x.strip()), sep=' ')
        
p = x.map(lambda s: ' '.join(s.split()))

In [27]:
dfs[0].PredictionString = p
print(len(dfs[0].iloc[0].PredictionString), len(dfs[1].iloc[0].PredictionString))

41432 5800


In [28]:
dfs[0].head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0001552 eNodkuuyqjAMRl+p8Xqq4gyIMFt...
1,52a76f3fe21185bf,1024,683,/m/04dr76w 0.0074179 eNqlU+ly2jAQfiVrdYAduykxD...
2,65e9da7c04c8e4cd,1024,682,/m/04yx4 0.0002104 eNpdkltzokAQhf/S9FAWbGVrH1L...
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0006262 eNpFiUEKgDAMBL+0SSuCHgRBpJ...
4,0841dee573a875ca,1024,683,/m/05r655 0.0010084 eNp1kutOAjEQhV/pdLfcjMaoQZ...


In [29]:
dfs[0].to_csv('concat_od275_htc4619_parent0338.csv', index=False)

In [30]:
!kaggle competitions submit -c open-images-2019-instance-segmentation -f concat_od275_htc4619_parent0338.csv -m "submit"

100%|██████████████████████████████████████| 4.54G/4.54G [02:23<00:00, 34.0MB/s]
Successfully submitted to Open Images 2019 - Instance Segmentation

In [ ]:
!ls

In [11]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

10336

In [56]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

0

In [12]:
len(dfs[1].loc[dfs[1].PredictionString.isna()])

41

In [100]:
#####
lst = ['Geeks', 'For', 'Geeks', 'is',  
            'portal', 'for', 'Geeks'] 
tmp1 = pd.DataFrame(lst, columns=['t'])
lst2 = ['a', 'b', 'c', 'd',  
            'e', 'f', 'g'] 
tmp2 = pd.DataFrame(lst2, columns=['t'])

lst3 = ['a1', 'b1', 'c', 'd',  
            'e', 'f', 'g'] 
tmp3 = pd.DataFrame(lst3, columns=['t'])
tmp3.head()

,t
0,a1
1,b1
2,c
3,d
4,e


In [4]:
df_is = pd.read_csv('ensemble_is_42_0808_1_3_models.csv')
df_is.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.1689330 /m/04bcr3 0.3664692 0.7250217 0.4814...
1,763c72626bf24534,0.7671249 /m/04bcr3 0.6873273 0.8193369 0.9977...
2,7540c162180973b1,0.0219995 /m/04dr76w 0.0762036 0.3234025 0.120...
3,c4e522dce52cb02f,0.4940703 /m/04bcr3 0.8307936 0.6809913 0.9920...
4,cafcd9496c808085,0.0129044 /m/04dr76w 0.0000000 0.0600080 0.031...


In [5]:
def get_low_conf_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return pred_str
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    
    res = []
    for i, c in enumerate(det_items):
        if i % 12 == 0:
            res.append(str(float(c)/4))
        else:
            res.append(c)
    return ' '.join(res)